In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import pickle
import pandas as pd
from tasks import wsd
from pathlib import Path
from tasks import wsd
from utils import nlp_tools
from tqdm.auto import tqdm
import numpy as np
import json
from sklearn.metrics import classification_report
from flair.embeddings import TransformerWordEmbeddings
from utils.dataset_download import harvest_data_from_extended_senses
from utils.classificaton_utils import binarize, vectorize_target_expressions,cosine_similiarity,eval_lemma

In [3]:
lemma = 'machine'
pos = 'NN'
senses = {'machine_nn01-38474140'} # machine_nn01-38475772 machine_nn01-38475923 machine_nn01-38475835 machine_nn01-38474140
relations = ['seed','synonym'] # ,'descendant','sibling'
eval_mode = "lemma_etal" # lemma or lemma_etal
    

In [4]:
df_train, df_val, df_test = binarize(lemma,
                        pos,
                        senses, 
                        relations,
                        strict_filter=True,
                        start=1700,
                        end=2000,
                        eval_mode=eval_mode)

# senses before filtering by date = 517
# senses after filtering by date = 433


# of seed senses 26 
# of synonyms 383 
# of branch senses 0


# of seeds selected 1 
# of synonyms selected 44 
# of branches selected 0
[LOG] #rows before removing None vector (1947, 21)
[LOG] #rows after removing None vector (1911, 21)


In [6]:
df_train.head(3)

,sense_id,lemma_definition,definition,word_id,lemma,quotation_id,source,text,year,full_text,...,keyword_offset,"vector_bert_base_-1,-2,-3,-4_mean","vector_blert_-1,-2,-3,-4_mean",label,id,daterange,provenance,provenance_type,relation_to_core_senses,relation_to_seed_senses
0,body_nn01-17170653,The complete physical form of a person or anim...,Particular technical uses. The part of a vehic...,body_nn01,body,body_nn01-132916428,"{'title': 'Material Handling Engin.', 'author'...","{'keyword': 'bodies', 'full_text': 'After car ...",1990.0,"After car bodies are painted, they are moved i...",...,10.0,"[1.2747291, 0.25178745, 0.69486666, 0.42832682...","[1.5054287, 1.1386966, 1.3405375, 0.8012274, -...",0,NaN,NaN,NaN,NaN,NaN,NaN
1,man_nn01-110482153,An adult male human being. Without explicit co...,"As vocative or as int., introducing a remark o...",man_nn01,man,man_nn01-110482440,"{'title': 'Shaela', 'author': 'R. Bulter', 'ge...","{'keyword': 'Min', 'full_text': 'Min A'm vexed...",1976.0,Min A'm vexed ta hear yun.,...,0.0,"[-0.10557328, 0.24347349, 0.731555, -0.4305202...","[-0.49209523, 0.7658461, 0.07512934, 0.0148925...",0,NaN,NaN,NaN,NaN,NaN,NaN
2,body_nn01-17169813,The complete physical form of a person or anim...,Contrasted with the soul. Cf. soul body n. at ...,body_nn01,body,body_nn01-17169857,"{'title': 'Ess. Man', 'author': 'A. Pope', 'ge...","{'keyword': 'Body', 'full_text': 'All are but ...",1733.0,"All are but parts of one stupendous Whole, Who...",...,49.0,"[0.8197431, 0.04237363, 0.6312159, -0.2658673,...","[0.60478234, 0.58020014, 0.053836707, -0.06571...",0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_train.columns

Index(['sense_id', 'lemma_definition', 'definition', 'word_id', 'lemma',
       'quotation_id', 'source', 'text', 'year', 'full_text', 'keyword',
       'keyword_offset', 'vector_bert_base_-1,-2,-3,-4_mean',
       'vector_blert_-1,-2,-3,-4_mean', 'label', 'id', 'daterange',
       'provenance', 'provenance_type', 'relation_to_core_senses',
       'relation_to_seed_senses'],
      dtype='object')

In [30]:
def enclose_keyword(row,enclose_token='[TARGET]'):
    """enclose keyword with specific token to point
    learner towards to word it has to focus on
    """
    sentence = ''
    for i,c in enumerate(row.full_text):
        if i == int(row.keyword_offset):
            sentence+=enclose_token + ' '
        elif i ==int(row.keyword_offset + len(row.keyword)):
            sentence+= ' ' + enclose_token
        sentence+=c
    return sentence

def merge_quotation_gloss(row):
    out_string = '[GLOSS] '
    if row.definition:
        out_string+=row.definition
    out_string+=' [QUOT] '  
    if row.enclosed_quotation:
        out_string+=row.enclosed_quotation
    return out_string

def merge_quotation_keyword(row):
    out_string = '[TARGET] '
    if row.keyword:
        out_string+=row.keyword
    out_string+=' [QUOT] '  
    if row.enclosed_quotation:
        out_string+=row.enclosed_quotation
    return out_string




In [31]:
from pathlib import Path
path = Path('./data/training_data')
path.mkdir(exist_ok=True)

In [35]:
csv_out_path = path / f"{lemma}_{'_'.join(senses)}"
csv_out_path.mkdir(exist_ok=True)


In [47]:
df_train['enclosed_quotation'] = df_train.apply(enclose_keyword, axis=1)
df_train['train_text'] = df_train.apply(merge_quotation_keyword, axis=1)
df_train[['train_text','label']].to_csv(csv_out_path / "train.csv",index = False, sep='\t')    
df_val['enclosed_quotation'] = df_val.apply(enclose_keyword, axis=1)
df_val['train_text'] = df_val.apply(merge_quotation_keyword, axis=1)
df_val[['train_text','label']].to_csv(csv_out_path / "dev.csv",index = False, sep='\t')        
df_test['enclosed_quotation'] = df_test.apply(enclose_keyword, axis=1)
df_test['train_text'] = df_test.apply(merge_quotation_keyword, axis=1)
df_test[['train_text','label']].to_csv(csv_out_path / "test.csv",index = False, sep='\t')    

In [49]:
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

# this is the folder in which train, test and dev files reside
data_folder = csv_out_path

# column format indicating which columns hold the text and label(s)
column_name_map = {0: "text", 1: "label"}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True,
                                         delimiter='\t',    # tab-separated files
) 

2021-01-14 16:01:53,769 Reading data from data/training_data/machine_machine_nn01-38474140
2021-01-14 16:01:53,770 Train: data/training_data/machine_machine_nn01-38474140/train.csv
2021-01-14 16:01:53,770 Dev: data/training_data/machine_machine_nn01-38474140/dev.csv
2021-01-14 16:01:53,770 Test: data/training_data/machine_machine_nn01-38474140/test.csv


In [50]:
# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()
print(label_dict)

2021-01-14 16:01:56,791 Computing label dictionary. Progress:
100%|██████████| 1604/1604 [00:00<00:00, 2103.07it/s]2021-01-14 16:01:57,857 [b'0', b'1']
Dictionary with 2 tags: 0, 1



In [ ]:
from torch.optim.adam import Adam
from flair.data import Corpus
from flair.datasets import TREC_6
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer




# 3. initialize transformer document embeddings (many models are available)
document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)

# 4. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

# 5. initialize the text classifier trainer with Adam optimizer
trainer = ModelTrainer(classifier, corpus, optimizer=Adam)

# 6. start the training
trainer.train('models/taggers/trec',
              learning_rate=1e-5, # use very small learning rate
              mini_batch_size=16,
              mini_batch_chunk_size=4, # optionally set this if transformer is too much for your machine
              max_epochs=5, # terminate after 5 epochs
              )